In [181]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit
import random as rd
import math
from qiskit import algorithms
from qiskit.circuit import Parameter
from qiskit.opflow import X,Z,I,CX,CZ
SPSA = algorithms.optimizers.SPSA #choix de l'optimiseur avant la routine VQE 
from qiskit import Aer
from qiskit.algorithms import eval_observables
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import CRXGate
from qiskit.opflow.primitive_ops import PrimitiveOp
from qiskit.opflow import CircuitStateFn
from qiskit.algorithms import VQE
from qiskit.opflow import commutator
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import SLSQP
seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

In [182]:
def ansatz(N): 
    qc = QuantumCircuit(N)
    for k in range(N):
        thet = rd.random()*2*math.pi
        qc.rx(thet,k)
    return(qc)

def measure_energy(qc, operator): 
    psi = CircuitStateFn(qc)
    return(psi.adjoint().compose(op).compose(psi).eval().real)

def trans(gate, n, i):
    #Envoie le "gate" de OperatorBase sur le qbit i, dont n est le nb total
    k = gate.num_qubits
    return(I.tensorpower(max(n-i-k, 0)) ^ gate ^ I.tensorpower(i))

def L_ops():
    #select operators from pool
    n = N//2
    L_c = []
    OPS1 = OPS.copy()
    for k in range(n):
        j = int(rd.random()*len(OPS1))
        L_c.append(OPS1[j])
        del OPS1[j]
    return(L_c)
    
def perm(k):
    "ici k est une liste de deux entiers distincts compris entre 0 et N-1"
    res = [i for i in range(N)] #on créée l'ordre identité
    p0 = k[0]
    p1 = k[1]
    res[p1], res[p0 + 1] = p0 + 1, p1
    return(res)

def Next_op(L_grad, e): 
    kmax = 0
    end = False 
    for k in range(0,len(L_grad)):
        if L_grad[k][1] > L_grad[kmax][1] :
            kmax = k
    if L_grad[kmax][1] < e : 
        end = True 
    print(L_grad[kmax][1])
    return(end,L_grad[kmax])

noms_p = []
for k in range(10000):
    noms_p += [Parameter('theta_'+str(k))]


In [183]:
#définition de l'hamiltonien

N = 4
J = 1
h = 3

"Hamiltonien pour 2 qbits:"

H = - J * CX.compose(Z.compose(CX)) - h * (X ^ I)

#Définition de l'hamiltonien du système
def hi(N, i):
    #n = nombre de qubits
    #i = indice
    if i >= N-1:
        return('Error Hi: The number of qubits is too small')
    H = (- J * CX.compose(Z.compose(CX))) - (h * (X ^ I))
    if i != 0:
        H = I.tensorpower(i) ^ H
    if i + 1 == N-1:
        return(H)
    else:
        H = H ^ I.tensorpower(N-i-2)
    return(H)


hamil = hi(N, 0)
for i in range(1, N-1):
    hamil += hi(N, i)


In [184]:
# notre pool d'opérateurs est initialisé selon la méthode suivante 
OPS = []    
for k in range(N): 
    OPS.append( [True,k])
for k in range(N): 
    for j in range(N):
        OPS.append([False,[k,j]])

In [185]:
def gradient(ansatz, hamil):
    
    list = []
    
    L_c = L_ops()
    
    for k in L_c : 
        
        if k[0] : 
            
            operator = trans(X,N,k[1])
            Com = commutator(hamil,operator)
            Avg = measure_energy(ansatz,Com)
            list.append([k,Avg])
        
        else : 
            
            operator = trans(CX, N, k[1][0]).permute(perm(k[1])).compose(trans(Z,N,k[1][0])).compose(trans(CX, N, k[1][0]).permute(perm(k[1])))
            
            Com = commutator(hamil,operator)
            
            Avg = measure_energy(ansatz, Com)
            
            list.append([k,Avg])
            
    return(list)

def grow_ansatz(Ansatz, operator, parameters):
    n = len(parameters)
    #parameters+=[noms_p[n]]
    k = operator
    if k[0]:
        Ansatz.rx(noms_p[n],k[1])
    else: 
        Ansatz.cx(k[1][0], k[1][1])
        Ansatz.rz(noms_p[n],k[1][0])
        Ansatz.cx(k[1][0], k[1][1])
    return(Ansatz, parameters)

def Vqe(Ansatz,Hamil): 
    seed = 50
    algorithm_globals.random_seed = seed
    qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
    slsqp = SLSQP(maxiter=1000)
    vqe = VQE(qc, optimizer=slsqp, quantum_instance=qi)
    result = vqe.compute_minimum_eigenvalue(operator= PrimitiveOp(Operator(hamil)))
    print(result)
    optimizer_evals = result.optimizer_evals
    return(result.optimal_parameters)
        

In [186]:
N = 4
Ansatz = ansatz(4)
Gradient = 1000
e = 0.01
parameters = []

while Gradient >= e :
    
    l = gradient(Ansatz, hamil) #step 1 calculation of the mean values of the commutators of the chosen pool operators with H
    
    Nop = Next_op(l,e)
    
    if Nop[0]:
        
        break 
    
    grow_ansatz(Ansatz, Nop[1], parameters)
    
    parameters+=[noms_op[len(parameters)]]
    
    opt_parameters = Vqe(Ansatz, Hamil)
    
    Ansatz = Ansatz.bind_parameters(opt_parameters)
    

    
    
    
    
    

-1.37e-16


In [135]:
qc = ansatz(4)
qc.rx(noms_p[0],1)

In [137]:


seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

slsqp = SLSQP(maxiter=1000)
vqe = VQE(qc, optimizer=slsqp, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator= PrimitiveOp(Operator(hamil)))
print(result)
optimizer_evals = result.optimizer_evals


{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 10,
    'eigenstate': array([ 2.47153558e-05-3.02675814e-21j, -1.69910419e-21-1.38742386e-05j,
       -1.49359862e-17-2.43923166e-01j, -1.36928970e-01+8.38448122e-18j,
       -5.90619226e-21-4.82277198e-05j, -2.70731644e-05+3.31550641e-21j,
       -4.75973650e-01+2.91449804e-17j,  1.63608574e-17+2.67193077e-01j,
        3.89761763e-21+3.18264632e-05j,  1.78661374e-05-2.18797080e-21j,
        3.14104791e-01-1.92333714e-17j, -1.07968659e-17-1.76326202e-01j,
        6.21038095e-05-7.60552315e-21j, -4.26944461e-21-3.48626609e-05j,
       -3.75305803e-17-6.12920889e-01j, -3.44069926e-01+2.10682067e-17j]),
    'eigenvalue': (-0.20784022689169585+0j),
    'optimal_circuit': None,
    'optimal_parameters': {Parameter(theta_0): 4.0575650549409294},
    'optimal_point': array([4.05756505]),
    'optimal_value': -0.20784022689169585,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': 0.07729315757751465

In [150]:
result.optimal_parameters[noms_p[0]]

4.0575650549409294

In [154]:
qc = qc.bind_parameters(result.optimal_parameters)

In [155]:
qc.draw()

┌───────────┐                         
q_0: ┤ Rx(1.023) ├─────────────────────────
     ├───────────┤ ┌──────────────────────┐
q_1: ┤ Rx(5.367) ├─┤ Rx(4.05756505494093) ├
     ├───────────┴┐└──────────────────────┘
q_2: ┤ Rx(2.1944) ├────────────────────────
     ├────────────┤                        
q_3: ┤ Rx(4.4622) ├────────────────────────
     └────────────┘